In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import sys
sys.path.append('../../PyCurb/')
sys.path.append('..')

In [5]:
import os
import json
from os.path import join
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime

from ccd.utils import read_layers_from_gdb, day_range
from ccd.rule import rule_parking_time
from ccd.parking_time_range import ParkingTimeRange

from curblr import (CurbLRObject, Regulation, Location, Feature, FeatureCollection, 
                    TimeSpan, Rule, UserClass, Payment, Manifest, Authority)
from curblr.utils import parse_time, to_camelcase, from_camelcase
from curblr.time_rule import DaysOfWeek, TimeOfDay, DesignatedPeriod

In [6]:
def row_to_ptrs(r):
    ranges = []
    for i in range(1,3):
        dr = day_range(r['Day_From_{}'.format(i)], r['Day_To_{}'.format(i)])
        ptr = ParkingTimeRange(rule_parking_time(r['Time_From_{}'.format(i)]), rule_parking_time(r['Time_To_{}'.format(i)]))
        if not dr and ptr.all_time:
            ranges.append(None)
        else:
            ranges.append((dr, ptr))
    
    return tuple(ranges)

In [7]:
geodatabase = '../data/raw/ParkingZones.gdb/'
layers = read_layers_from_gdb(geodatabase)
pz = layers['Parking_Zones'].to_crs({'init':'epsg:4326'})
pzr = layers['Parking_Zone_Regulations']

In [8]:
with open('../data/interim/parking_data/output_30_side-of-street_hierarchy-8_best-direction.matched.geojson') as f:
    lrm = json.load(f)
lrms = {f['properties']['pp_globalid']: f for f in lrm['features']}

In [9]:
with open('../data/interim/reg_lookup.json') as f:
    d = json.load(f)

In [13]:
phila = Authority('ppa', 'http://www.philapark.org/', '1-888-591-3636')
manifest = Manifest(time_zone='EET', currency='USD', authority=phila)
fc = FeatureCollection(manifest=manifest)

for i, seg in pz.iterrows():
    gid = seg['GlobalID']
    iid = gid.replace('{', '').replace('}', '')
    
    images = [join('img', i) for i in os.listdir('../img/') if iid in i]
    
    # initialize the feature
    lrm = lrms.get(gid)
    if lrm:
        lrm['properties']['marker'] = 'sign'
    loc = Location.from_lr_feature(lrm, 
                                   object_id='pp_globalid', 
                                   street_name='pp_street')
    feature = Feature(seg['geometry'], loc, images=images)
    
    # get all regulations for this segment
    regs_gdf = pzr[pzr['GUID'] == gid]
    
    # payment
    m = seg['Metering']
    payment = Payment(methods = [m.lower().strip()]) if m not in (None, 'No Meters') else None
    
    # regulations
    if len(regs_gdf) == 0:
        reg_obj = Regulation(Rule('parking', 'no regulations listed'))
        feature.add_regulation(reg_obj)
    else:
        for _, r in regs_gdf.iterrows():
            reg_c = r['Regulation'].lower().strip()
            if reg_c == 'other (see notes)':
                reg_c = r['Notes']
                if reg_c:
                    reg_c = reg_c.lower().strip()
                else:
                    reg_c = 'no regulations listed'

            # get mapping of reg string to other characteristics
            s = d.get(reg_c)

            # priority
            priority = s.get('priority', 4)

            # rule
            max_stay = r['Time_Limit'] * 60 if not np.isnan(r['Time_Limit']) else None
            clr_rule = Rule(s['activity'], s['reason'], max_stay=max_stay)

            clr_userclasses = None
            classes = [c for c in s.get('classes') if c]

            if len(classes) > 0:
                clr_userclasses = [UserClass(classes)]
        
            # timespans
            timespans = []
            for p in row_to_ptrs(r):
                if p:
                    dr, ptr = p
                    if dr:
                        if len(dr) < 7:
                            days_of_week = DaysOfWeek(dr)
                        else:
                            days_of_week = None
                    else: 
                        days_of_week = None

                    if ptr.all_time or ptr.invalid:
                        time_of_day = None
                    else:
                        time_of_day = TimeOfDay(ptr.start.hour, ptr.end.hour)

                    # designated period
                    designated_period = None
                    dp = s.get(designated_period)
                    if dp:
                        designated_period = DesignatedPeriod(**dp)
                    elif r['Time_From_1'] == 9910:
                        designated_period = DesignatedPeriod('school hours', 'only during')
                    designated_period = []
                    
                    ts = TimeSpan(days_of_week=days_of_week, 
                                  times_of_day=[time_of_day] if time_of_day else time_of_day, 
                                  designated_periods=[designated_period] if designated_period else designated_period)

                    new_ts = True
                    for t in timespans:
                        if not ts.days_of_week:
                            if not t.days_of_week:
                                continue
                        elif t.days_of_week: 
                            if sorted(ts.days_of_week.days) != sorted(t.days_of_week.days):
                                continue

                        for tod in ts.times_of_day:
                            t.add_time_of_day(tod)
                        new_ts = False

                    if new_ts:
                        timespans.append(ts)

            # generate regulation
            reg_obj = Regulation(clr_rule, 
                                 clr_userclasses, 
                                 timespans, 
                                 payment=payment,
                                 priority=priority)

            feature.add_regulation(reg_obj)
    
    fc.add_feature(feature)
    
    if (i + 1) % 500 == 0:
        print('Completed [{}] of {}'.format(i+1, len(pz)))
print('Completed [{}] of {}'.format(i+1, len(pz)))

Completed [500] of 3349
Completed [1000] of 3349
Completed [1500] of 3349
Completed [2000] of 3349
Completed [2500] of 3349
Completed [3000] of 3349
Completed [3349] of 3349


In [14]:
fc.save('../data/interim/ccd_curblr.json', add_timestamp=True)

In [15]:
lrm

{'type': 'Feature',
 'properties': {'shstReferenceId': '165d66a82288eb51c256489f72f63c67',
  'shstGeometryId': 'bf26c0f927c74e5d633386632d60f8cd',
  'shstFromIntersectionId': '99962b850f699e0e6b77ea1073424a63',
  'shstToIntersectionId': '980fad0f60037bb54cf98b8c72641ee7',
  'referenceLength': 55.6,
  'section': [0, 17.328605466924394],
  'gisReferenceId': 'cb1b89431c7d3c068e6c9ca1bdf75653',
  'gisGeometryId': 'a7674675a2414f7a7e046d4a95479134',
  'gisTotalSegments': 2,
  'gisSegmentIndex': 2,
  'gisFromIntersectionId': 'e833396ea69a408ed51ff5059a0a50d3',
  'gisToIntersectionId': 'f240d9c8999b134c734518f380e0d169',
  'startSideOfStreet': 'left',
  'endSideOfStreet': 'left',
  'sideOfStreet': 'left',
  'score': 6.31,
  'matchType': 'hmm',
  'pp_globalid': '{29428B3E-3905-401D-BE9E-838559563A11}',
  'pp_metering': 'No Meters',
  'pp_permitzone1': None,
  'pp_permitzone2': None,
  'pp_permitexception': 'No',
  'pp_towzone': 'No',
  'pp_street': 'S INDEPENDENCE MALL E',
  'pp_block': '100',